# Sprint 13 - Proyecto

#### Cargar las librerías

In [4]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    auc,
    f1_score,
    mean_squared_error
)

# Utilities
from sklearn.utils import shuffle


In [8]:
#Cargar los archivos

gold_recovery_train = pd.read_csv('data/gold_recovery_train.csv')
gold_recovery_test = pd.read_csv('data/gold_recovery_test.csv')
gold_recovery_full = pd.read_csv('data/gold_recovery_full.csv')